In [5]:
import requests

KAKAO_API_KEY = 'api 키'

def get_places_nearby(lat, lng, radius=1000, query='랜드마크'):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {
        "query": query,
        "y": lat,
        "x": lng,
        "radius": radius,
        "size": 10
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        places = response.json().get('documents', [])
        return [(place['place_name'], float(place['y']), float(place['x'])) for place in places]
    else:
        print("Error:", response.status_code, response.text)
        return []
    
user_lat, user_lng = 37.5665, 126.9780 

places = get_places_nearby(user_lat, user_lng, query="공원")
print("Nearby Places:", places)


Nearby Places: [('서울광장', 37.5656726042805, 126.977968926957), ('청계광장', 37.569074693516065, 126.97759212096156), ('청계천한빛광장', 37.5680487407066, 126.985067655744), ('세종로공원', 37.5736766421518, 126.975892728114), ('숭례문광장', 37.5598984799777, 126.975249777876), ('광화문원표공원', 37.568865479155, 126.976645884677), ('서소문역사공원', 37.56128103287903, 126.96937290180193), ('정동공원', 37.567650747545706, 126.97166469208234), ('청진공원', 37.5714032803741, 126.979896124362), ('세종문화회관 예술의정원', 37.5722227441159, 126.97520381645698)]


In [7]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

user_preferences = [
    [37.5665, 126.9780],  
    [37.5700, 126.9830],  
    [37.5500, 126.9900], 
]

candidate_places = np.array([
    [37.5741, 126.9840],  
    [37.5605, 126.9827], 
    [37.5582, 126.9875],  
])

knn = NearestNeighbors(n_neighbors=2, algorithm='ball_tree')
knn.fit(user_preferences)

distances, indices = knn.kneighbors(candidate_places)

recommendations = [candidate_places[idx] for idx in indices[0]]
print("Recommended Places:", recommendations)

Recommended Places: [array([ 37.5605, 126.9827]), array([ 37.5741, 126.984 ])]


In [9]:
def recommend_route(user_lat, user_lng):
    """
    사용자의 위치와 주변 장소를 기반으로 추천 장소를 반환합니다.
    - user_lat, user_lng: 사용자 위치 (위도, 경도)
    - 반환값: 추천 장소 리스트 [(이름, 위도, 경도), ...]
    """
    nearby_places = get_places_nearby(user_lat, user_lng, query="공원")

    user_preferences = np.array([
        [36.353, 128.699],  
        [36.355, 128.702],  
        [36.350, 128.695],  
    ])

    candidate_places = np.array([[place[1], place[2]] for place in nearby_places])

    if len(candidate_places) == 0:
        print("근처에 추천할 장소가 없습니다.")
        return []

    knn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
    knn.fit(user_preferences)

    distances, indices = knn.kneighbors(candidate_places)

    seen_places = set() 
    recommended_places = []

    for idx in indices.flatten():
        place = nearby_places[idx]
        if place[0] not in seen_places:  
            recommended_places.append(place)
            seen_places.add(place[0])

    return recommended_places

user_lat, user_lng = 36.3524, 128.6976  
recommended = recommend_route(user_lat, user_lng)

print("Recommended Route:")
for place in recommended:
    print(f"장소 이름: {place[0]}, 위치: ({place[1]}, {place[2]})")

Recommended Route:
장소 이름: 호국동산, 위치: (36.357934364153, 128.702469266597)
장소 이름: 구봉공원, 위치: (36.3476526647845, 128.691982261711)
장소 이름: 의성키움어린이공원, 위치: (36.3501917686313, 128.702011018671)
